## Load Dependencies

In [1]:
!pip install -q llmlingua

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00


In [3]:
#import libraries
import json
import os

from llmlingua import PromptCompressor
from openai import OpenAI
from google.colab import userdata

#provide openai API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY4")

In [4]:
#openai client
client = OpenAI()

#prompt compressor
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    use_llmlingua2=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

## Sample Data

In [5]:
context = """
Natural Language Processing(NLP) is a subset of Artificial intelligence which involves communication between a human and a machine using a natural language than a coded or byte language. It provides the ability to give instructions to machines in a more easy and efficient manner.

Natural Language Understanding(NLU) is an area of artificial intelligence to process input data provided by the user in natural language say text data or speech data. It is a way that enables interaction between a computer and a human in a way like humans do using natural languages like English, French, Hindi etc.

Natural Language Generation(NLG) is a sub-component of Natural language processing that helps in generating the output in a natural language based on the input provided by the user. This component responds to the user in the same language in which the input was provided say the user asks something in English then the system will return the output in English.
"""
question = "what is the different between NLU and NLG ?"

prompt = "\n\n".join([context, question])

## Before Prompt Compression

In [6]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

In [7]:
response.choices[0].message.content

'Natural Language Understanding (NLU) and Natural Language Generation (NLG) are both subfields of Natural Language Processing (NLP), but they serve different purposes within the realm of human-computer interaction.\n\n1. **Natural Language Understanding (NLU):**\n   - **Purpose:** NLU focuses on interpreting and making sense of the input provided by users in natural language form, such as spoken or written text.\n   - **Functionality:** It involves tasks like identifying the intent of the user, extracting entities, managing context, and handling ambiguities in language. The goal is to understand what the user means or is asking for.\n   - **Example Applications:** Virtual assistants understanding user requests, sentiment analysis, topic detection.\n\n2. **Natural Language Generation (NLG):**\n   - **Purpose:** NLG is concerned with creating natural language text or speech based on data or structured information.\n   - **Functionality:** It involves selecting the appropriate content to 

In [8]:
response.usage

CompletionUsage(completion_tokens=317, prompt_tokens=194, total_tokens=511, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

## After Prompt Compression

In [9]:
compressed_prompt = llm_lingua.compress_prompt(
    context,
    rate=0.33,
    force_tokens=["!", ".", "?", "\n"],
    drop_consecutive=True,
)

In [10]:
compressed_prompt = "\n\n".join([compressed_prompt["compressed_prompt"], question])

response_compressed = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": compressed_prompt}]
)

In [11]:
response_compressed.choices[0].message.content

"Natural Language Understanding (NLU) and Natural Language Generation (NLG) are both components of Natural Language Processing (NLP). The difference between the two lies in their functions:\n\n1. NLU is about the machine's ability to understand and interpret human language or input. It focuses on machine reading comprehension, enabling an AI to understand idioms, context, and overall semantic and sentiment.\n\n2. On the other hand, NLG is about generating human-like language or output. The aim here is to produce content or responses that echo the way humans write or speak. It's about converting structured data into human-readable and understandable text."

In [12]:
response_compressed.usage

CompletionUsage(completion_tokens=128, prompt_tokens=80, total_tokens=208, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [13]:
(response.usage.total_tokens - response_compressed.usage.total_tokens) / response.usage.total_tokens * 100

59.295499021526425

- the reductions is massive around 59% while still maintaining the accurate predictions